In [4]:
import nest_asyncio
nest_asyncio.apply()


from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import asyncio
from langdetect import detect
from gensim.models import KeyedVectors, Word2Vec
from gensim.test.utils import common_texts
import gensim.downloader as api


In [5]:
#%ProgramFiles%\Google\Chrome\Application\chrome.exe


#setting up webdriver
webdriver_path = '%ProgramFiles%\Google\Chrome\Application\chrome.exe'


#chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
#chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)


#loggin in 
login_url = 'https://www.goodreads.com/ap/signin?language=en_US&openid.assoc_handle=amzn_goodreads_web_na&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fsign-in&siteState=ba99ed98e3cb4430399c0bc24e911cce'
driver.get(login_url)
#seeing if already logged in. 
try: 
    driver.find_element(by.CLASS_NAME, 'gr-h3 gr-h3--noTopMargin')
    print('already logged in, skipping login proccess')
#fuck this
except: 
    #looking for where to login 
    username_field = driver.find_element(By.NAME, "email")
    password_field = driver.find_element(By.NAME, "password")
    #logging in
    username_field.send_keys('helloworldtesting59@gmail.com')
    password_field.send_keys('securepassword')
    password_field.send_keys(Keys.RETURN)
    time.sleep(1)
    
    
#beautifulsoup addiiton
url = 'https://www.goodreads.com/shelf/show/non-fiction'
page_number = 1


#creating lists
title_list = []
author_list = []
avgrating_list = []
number_of_ratings = []
pub_date_list = []
review_list = []
synopsis_list = []
genre_list = []
    
#page numbers 
while page_number <= 1:
    
    #beautifulSoup 
    page = driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #what im tryna extract
    next_page = soup.find('a', class_ = 'next_page')
    titles = soup.find_all('a', class_ = 'bookTitle')
    author_name = soup.find_all('a', class_ ='authorName')
    avg_rating = soup.find_all('span', class_ ='greyText smallText')
    
    
    #checking to see if I got what i need
    for title, author, average_rating in zip(titles, author_name, avg_rating):
        #breaking down the avg_rating into rating list, rating avg,number of rating, publish date 
        clean_list = average_rating.get_text().replace('\n', '').strip()
        result_re = re.split(r'\s*—\s*', clean_list)
        
        
        title_list.append(title.get_text())
        author_list.append(author.get_text())
        avgrating_list.append(float(result_re[0].split()[2]))
        number_of_ratings.append(int(result_re[1].split()[0].replace(',','')))
        try:
            publication_date = int(result_re[2].split()[-1])
            pub_date_list.append(publication_date)
        except ValueError:
            pub_date_list.append('errorlololol')

    for title in titles:    
        #setting up restart loop
        max_retries = 3
        success = False
        #extracting reviews + synopsis
        for attempt in range(3):
            try: 
                book_page = 'https://www.goodreads.com/' + title['href']
                driver.get(book_page)
                bp_soup = BeautifulSoup(driver.page_source, 'html.parser')

                #reviews        
                reviews = bp_soup.find_all('section', class_='ReviewText__content')
                reviews_str = '!121!'.join([rev.get_text().strip() for rev in reviews if detect(rev.get_text()) == 'en'])
                review_list.append(reviews_str)

                # synopsis
                synopsis = bp_soup.find('span', class_='Formatted')
                synopsis_list.append(synopsis.get_text())

                # genres
                genre_section = bp_soup.find('div', class_='BookPageMetadataSection__genres')
                if genre_section:
                    genre_elements = genre_section.find_all('a')
                    genres = [genre.get_text(strip=True) for genre in genre_elements]
                    genre_list.append(genres)
                else: 
                    print('failed to extract genre info')
                    genre_list.append('errorlololol')

                success = True
                break

            except Exception as e: 
                print(f'bad link or error, {title.get_text()} because of {e}, retrying... ')
                time.sleep(2)

        if not success: 
            print(f'failed to retrieve data for {title.get_text()}  ')
            genre_list.append("errorlololol")
            synopsis_list.append('errorlololol')
            review_list.append('errorlolololol')

            time.sleep(0.6)


            
    #next page
    if next_page:
        url = 'https://www.goodreads.com/' + next_page['href']
        page_number += 1
        print(f'went to page {page_number}')
        
    else:
        
        break

driver.close()
    
        

KeyboardInterrupt: 

webscraping done!, from there need to check to make sure the length of all lists are the same so we can establish a df

In [ ]:
        
dictionary = {'Book Title': title_list, 
            'Author': author_list, 
            'Average Rating': avgrating_list,
            'Total Rating': number_of_ratings,
            'Publish Date': pub_date_list, 
            'Synopsis': synopsis_list,
            'Review': review_list, 
            'Genres': genre_list
            }

fixing data lol

In [ ]:
df = pd.DataFrame(dictionary)
df.to_csv('goodreads_nonfiction_books.csv', index=False)

#ARCHIVES